In [43]:
sc

In [44]:
def extractRow(index, lines):
    import csv
    if index == 0:
        lines.next()
        
    reader = csv.reader(lines)
    for row in reader:
        if len(row) == 9:
            yield row

In [45]:
rdd = sc.textFile("yelp_review.csv", use_unicode=False)

rdd2 = rdd.mapPartitionsWithIndex(extractRow)

rdd2.take(1)

[['vkVSCC7xljjrAI4UGfnKEQ',
  'bv2nCi5Qv5vroFiqKGopiw',
  'AEx2SYEUJmTxVVB18LlCwA',
  '5',
  '2016-05-28',
  "Super simple place but amazing nonetheless. It's been around since the 30's and they still serve the same thing they started with: a bologna and salami sandwich with mustard. Staff was very helpful and friendly.",
  '0',
  '0',
  '0']]

In [46]:
rdd3 = rdd2.map(lambda x: [x[0],x[1],x[2],x[3],x[4],x[5],x[6],x[7],x[8]])
#rdd3 = rdd2.map(lambda x: (x[0],x[1],x[2],x[3],x[4],x[5],x[6],x[7],x[8]))
rdd3.take(2)

[['vkVSCC7xljjrAI4UGfnKEQ',
  'bv2nCi5Qv5vroFiqKGopiw',
  'AEx2SYEUJmTxVVB18LlCwA',
  '5',
  '2016-05-28',
  "Super simple place but amazing nonetheless. It's been around since the 30's and they still serve the same thing they started with: a bologna and salami sandwich with mustard. Staff was very helpful and friendly.",
  '0',
  '0',
  '0'],
 ['n6QzIUObkYshz4dz2QRJTw',
  'bv2nCi5Qv5vroFiqKGopiw',
  'VR6GpWIda3SfvPC-lg9H3w',
  '5',
  '2016-05-28',
  "Small unassuming place that changes their menu every so often. Cool decor and vibe inside their 30 seat restaurant. Call for a reservation. We had their beef tartar and pork belly to start and a salmon dish and lamb meal for mains. Everything was incredible! I could go on at length about how all the listed ingredients really make their dishes amazing but honestly you just need to go. A bit outside of downtown montreal but take the metro out and it's less than a 10 minute walk from the station.",
  '0',
  '0',
  '0']]

In [47]:
rdd4 = rdd3.filter(lambda row: (row != None))
rdd4.take(1)

[['vkVSCC7xljjrAI4UGfnKEQ',
  'bv2nCi5Qv5vroFiqKGopiw',
  'AEx2SYEUJmTxVVB18LlCwA',
  '5',
  '2016-05-28',
  "Super simple place but amazing nonetheless. It's been around since the 30's and they still serve the same thing they started with: a bologna and salami sandwich with mustard. Staff was very helpful and friendly.",
  '0',
  '0',
  '0']]

In [49]:
rdd5 = rdd4.map(lambda row: (row[2],1)) #works for row[1]
rdd5.take(5)

[('AEx2SYEUJmTxVVB18LlCwA', 1),
 ('VR6GpWIda3SfvPC-lg9H3w', 1),
 ('CKC0-MOWMqoeWf6s-szl8g', 1),
 ('ACFtxLv8pGrrxMm6EgjreA', 1),
 ('s2I_Ni76bjJNK9yG60iD-Q', 1)]

In [50]:
from operator import add
#rdd6 = rdd5.reduceByKey(lambda x,y: x+y)
rdd6 = rdd5.reduceByKey(add)
rdd6.take(5)

[('Q39lkuSwfg-7CblOixLQDA', 51),
 ('rbDqCV2g23K3ZrTxmgoNBg', 141),
 ('h_-KUSA612pFEIGC_dCsKQ', 46),
 ('8mSLR5vjM1vnNXb6E6t1og', 7),
 ('dFA7C1lCg-FZvn15Q8nZYg', 13)]

In [54]:
# Create the schema necessary for the creation of a DataFrame
from pyspark.sql.types import *
schema = StructType([
    StructField("review_id", StringType(), False), 
    StructField("user_id", StringType(), False)])

In [55]:
df = sqlContext.createDataFrame(rdd6, schema)
df.show(4)

+--------------------+-------+
|           review_id|user_id|
+--------------------+-------+
|Q39lkuSwfg-7CblOi...|     51|
|rbDqCV2g23K3ZrTxm...|    141|
|h_-KUSA612pFEIGC_...|     46|
|8mSLR5vjM1vnNXb6E...|      7|
+--------------------+-------+
only showing top 4 rows



In [56]:
print(df.count())

174567


In [ ]:
#!/usr/bin/env python
'''
Color parts of a line based on its properties, e.g., slope.
'''
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm

x = np.linspace(0, 3 * np.pi, 500)
y = np.sin(x)
z = np.cos(0.5 * (x[:-1] + x[1:]))  # first derivative

# Create a colormap for red, green and blue and a norm to color
# f' < -0.5 red, f' > 0.5 blue, and the rest green
cmap = ListedColormap(['r', 'g', 'b'])
norm = BoundaryNorm([-1, -0.5, 0.5, 1], cmap.N)

# Create a set of line segments so that we can color them individually
# This creates the points as a N x 1 x 2 array so that we can stack points
# together easily to get the segments. The segments array for line collection
# needs to be numlines x points per line x 2 (x and y)
points = np.array([x, y]).T.reshape(-1, 1, 2)
segments = np.concatenate([points[:-1], points[1:]], axis=1)

# Create the line collection object, setting the colormapping parameters.
# Have to set the actual values used for colormapping separately.
lc = LineCollection(segments, cmap=cmap, norm=norm)
lc.set_array(z)
lc.set_linewidth(3)
plt.gca().add_collection(lc)

plt.xlim(x.min(), x.max())
plt.ylim(-1.1, 1.1)
plt.show()